In [4]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import os

import utilities

In [5]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteWQEI89\imdb_reviews-train.tfrecord*...…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteWQEI89\imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteWQEI89\imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [12]:
trn_txt, trn_lb = [], []
for x, lb in imdb['train']:
    trn_txt.append(x.numpy().decode('utf-8'))
    trn_lb.append(lb.numpy())
    
val_txt, val_lb = [], []
for x, lb in imdb['test']:
    val_txt.append(x.numpy().decode('utf-8'))
    val_lb.append(lb.numpy())
    
trn_lb = np.array(trn_lb)
val_lb = np.array(val_lb)

In [ ]:
def fit_imdb(vocab_size=10000, embed_d=16, maxlen=120)
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(trn_txt, oov_token='<OOV>')
    
    trn_seq = tokenizer.text_to_sequence(trn_txt)
    val_seq = tokenizer.text_to_sequence(val_txt)